## FTSE 100 - Moving Averages Crossover 

How long does it take moving averages to crossover?

**Strategy:**
1. Find the difference in days between the first time the moving average cross over and when they cross over again.
- Do this for the entire history of stock and for every stock in FTSE 100.
- Calculate the average.

In [62]:
import os

import cufflinks as cf
import numpy as np
import pandas as pd
from pandas_datareader import DataReader
import plotly.offline as pyo

pyo.init_notebook_mode(connected=True)

In [66]:
ftse100_symbols = os.listdir('../data/stocks/FTSE 100/')
ftse100_symbols[:10]

['EVR.L.csv',
 'AAL.L.csv',
 'SDR.L.csv',
 'HIK.L.csv',
 'NMC.L.csv',
 'FRES.L.csv',
 'CPG.L.csv',
 'HL.L.csv',
 'LSE.L.csv',
 'PRU.L.csv']

In [67]:
df = DataReader(name='CNA.L', data_source='yahoo', start='2015-01-01')
df.head()

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2015-01-02,281.299988,275.200012,280.600006,276.799988,7405007.0,189.781372
2015-01-05,278.256012,264.708008,275.500000,268.200012,12551313.0,183.885010
2015-01-06,269.980011,263.899994,268.799988,263.899994,13797383.0,180.936783
2015-01-07,267.494995,263.000000,265.299988,265.200012,12813169.0,181.828110
2015-01-08,272.373993,266.399994,267.000000,271.299988,14430078.0,186.010422


In [3]:
pyo.iplot(
    df['Close'].iplot(asFigure=True),
    # image='png',
    filename='ply_01'
)

In [71]:
short = 25
long = 200
df['close_ma_short'] = df['Close'].rolling(window=short).mean()
df['close_ma_long'] = df['Close'].rolling(window=long).mean()

In [72]:
pyo.iplot(
    df[['Close', 'close_ma_short', 'close_ma_long']].iplot(
        asFigure=True,
        title='Return of The Jedi',
        xTitle='date',
        yTitle='value',
#         mode={'Close': 'lines', 'b': 'lines+markers'},
#         symbol={'a': 'circle','b': 'diamond'},
        size=3.5,
#         colors={'a': 'blue', 'b': 'red'}
    )
)

In [6]:
df = DataReader('AAPL', data_source='yahoo')

In [7]:
sma1 = 42
sma2 = 252

In [8]:
df['sma1'] = df['Close'].rolling(sma1).mean()
df['sma2'] = df['Close'].rolling(sma2).mean()

In [9]:
pyo.iplot(
    df[['Close', 'sma1', 'sma2']].iplot(
        asFigure=True
    )
)

In [10]:
df = df.dropna()

In [11]:
df['position'] = np.where(df['sma1'] > df['sma2'], 1, -1)

In [12]:
df['returns'] = np.log(df['Close']/df['Close'].shift(1))

In [13]:
df['strategy'] = df['position'].shift(1) * df['returns']

In [14]:
df.head()

,High,Low,Open,Close,Volume,Adj Close,sma1,sma2,position,returns,strategy
Date,,,,,,,,,,,
2010-12-31,46.211430,45.901428,46.135715,46.080002,48377000.0,40.094254,45.278436,37.120351,1,NaN,NaN
2011-01-03,47.180000,46.405716,46.520000,47.081429,111284600.0,40.965591,45.347177,37.185862,1,0.021500,0.021500
2011-01-04,47.500000,46.878571,47.491428,47.327145,77270200.0,41.179398,45.410068,37.252137,1,0.005205,0.005205
2011-01-05,47.762856,47.071430,47.078571,47.714287,63879900.0,41.516247,45.463572,37.321882,1,0.008147,0.008147
2011-01-06,47.892857,47.557144,47.817142,47.675713,75107200.0,41.482681,45.520034,37.391695,1,-0.000809,-0.000809


In [15]:
df = df.dropna()

In [16]:
np.exp(df[['returns', 'strategy']].sum())

returns     6.326388
strategy    5.222697
dtype: float64

In [17]:
df[['returns', 'strategy']].std() * 252 ** 0.5

returns     0.257087
strategy    0.257151
dtype: float64

In [18]:
pyo.iplot(
    df[['returns', 'strategy']].cumsum().apply(np.exp).iplot(asFigure=True))

In [19]:
ax = df[['returns', 'strategy']].cumsum().apply(np.exp).plot(figsize=(10, 6))
df['position'].plot(ax=ax, secondary_y='position', style='--')
ax.get_legend().set_bbox_to_anchor((0.20, 0.85))

### Optimising Moving Averages

In [20]:
from itertools import product

In [48]:
df1 = DataReader('BARC.L', data_source='yahoo')

In [49]:
sma1 = range(20, 61, 4)
sma2 = range(180, 281, 10)

In [81]:
def optimal_sma_pair(symbol):
    """
    Finds the optimal long time and short time average days for
    simple moving average strategy.
    
    Takes a symbol e.g. AAPL and returns a dataframe with 
    market returns, strategy returns and the difference.
    """
    results = pd.DataFrame()
    data = DataReader(symbol, data_source='yahoo')
    for SMA1 in sma1:
        for SMA2 in sma2:
            df = data.copy()
            df.dropna(inplace=True)
            df['returns'] = np.log(df['Close']/df['Close'].shift(1))
            df['SMA1'] = df['Close'].rolling(SMA1).mean()
            df['SMA2'] = df['Close'].rolling(SMA2).mean()
            df.dropna(inplace=True)
            df['position'] = np.where(df['SMA1'] > df['SMA2'], 1, -1)
            df['strategy'] = df['position'].shift(1) * df['returns']
            perf = np.exp(df[['returns', 'strategy']].sum())
            results = results.append(
                pd.DataFrame(
                    {'SMA1': SMA1, 'SMA2': SMA2,
                     'MARKET': perf['returns'],
                     'STRATEGY': perf['strategy'],
                     'OUT': perf['strategy'] - perf['returns']},
                    index = [0]), ignore_index=True)
            
    return results.sort_values(by='OUT', ascending=False)

In [86]:
results = optimal_sma_pair('BARC.L')

In [87]:
results.head()

,SMA1,SMA2,MARKET,STRATEGY,OUT
1,20,190,0.649096,2.627865,1.978768
0,20,180,0.617853,2.568637,1.950785
11,24,180,0.617853,2.373290,1.755437
35,32,200,0.694658,2.132477,1.437819
59,40,220,0.682588,2.110210,1.427623


### Stock Price today vs yesterday

In [93]:
df['Close'].head()

Date
2015-01-02    276.799988
2015-01-05    268.200012
2015-01-06    263.899994
2015-01-07    265.200012
2015-01-08    271.299988
Name: Close, dtype: float64

In [94]:
df['close_today_yday_diff'] = df['Close'] - df['Close'].shift(1)

In [96]:
pyo.iplot(
    df['close_today_yday_diff'].iplot(asFigure=True)
)

In [97]:
import plotly.graph_objects as go
import datetime

x = [datetime.datetime(year=2013, month=10, day=4),
     datetime.datetime(year=2013, month=11, day=5),
     datetime.datetime(year=2013, month=12, day=6)]

fig = go.Figure(data=[go.Scatter(x=x, y=[1, 3, 6])])
# Use datetime objects to set xaxis range
fig.update_layout(xaxis_range=[datetime.datetime(2013, 10, 17),
                               datetime.datetime(2013, 11, 20)])
fig.show()

### Mock data plots

In [24]:
a = np.random.standard_normal((250, 5)).cumsum(axis=0)
index = pd.date_range('2019-01-01', freq='B', periods=len(a))
df = pd.DataFrame(a, index=index, columns=list('abcde'))

In [25]:
df.head()

,a,b,c,d,e
2019-01-01,-1.247722,-0.156769,0.643390,-1.131578,0.218386
2019-01-02,-2.445842,-0.804176,1.537323,-2.724687,1.361353
2019-01-03,-2.176661,-0.541258,2.094550,-3.016594,2.172297
2019-01-04,-2.948337,1.141585,1.909535,-3.138871,1.568195
2019-01-07,-3.106879,0.733801,3.659659,-1.688364,2.252521


In [26]:
plyo.iplot(
    df.iplot(asFigure=True),
    # image='png',
    filename='ply_01'
)

NameError: name 'plyo' is not defined

In [ ]:
plyo.iplot(
    df[['a', 'b']].iplot(
        asFigure=True,
        theme='polar',
        title='A Time Series Plot',
        xTitle='date',
        yTitle='value',
        mode={'a': 'markers', 'b': 'lines+markers'},
        symbol={'a': 'circle','b': 'diamond'},
        size=3.5,
        colors={'a': 'blue', 'b': 'red'}
    )
)